In [ ]:
import pandas as pd
import requests
from datetime import datetime
import time

In [ ]:
owner = 'NixOS'  
repo = 'nixpkgs' 

url = f'https://api.github.com/repos/{owner}/{repo}/commits'    #endpoint commits
response = requests.get(url)
response.status_code

In [ ]:
# get companies from GitHub, try with 1000 commits 

TOKEN = ''   #GitHub API Token

# Repository:
owner = 'NixOS'
repo = 'nixpkgs'
url = f'https://api.github.com/repos/{owner}/{repo}/commits?per_page=10'   # Limit at 10 commits per request

headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

# initialze dataframe 
commit_data = pd.DataFrame(columns=['Email', 'Timestamp', 'Company'])  

commit_limit = 1000  # limit 
commit_count = 0

# commit function
def get_commits(url):
    global commit_count
    global commit_data

    while url and commit_count < commit_limit:
        response = requests.get(url, headers=headers)
        
        try:
            response.raise_for_status()
            commits = response.json()

            for commit in commits:
                if commit_count >= commit_limit:
                    break
                author_url = commit['author']['url'] if commit['author'] else None
                email = commit['commit']['author']['email']
                timestamp = commit['commit']['author']['date']
                company = None

                # if author_url: vorhanden ist, try and get profilinfo for comopanyname
                if author_url:
                    user_response = requests.get(author_url, headers=headers)
                    if user_response.status_code == 200:
                        user_data = user_response.json()
                        company = user_data.get('company')  

                # append in df
                commit_data.loc[commit_count] = [email, timestamp, company]
                commit_count += 1

            # check next page
            url = response.links.get('next', {}).get('url')

            # waiting time between requests
            time.sleep(1)

        except requests.exceptions.HTTPError as err:
            print(f'HTTP Error: {err}')
            break


get_commits(url)

commit_data

In [ ]:
commit_data['Company'] = commit_data['Company'].str.replace('@', '', regex=False) #performs simple, direct string replacement(default r=True)
commit_data['Company'] = commit_data['Company'].str.strip()
commit_data['Company'].unique()

In [ ]:
# getting commitinfos for 2024

TOKEN = ''
# Repository
owner = 'NixOS'
repo = 'nixpkgs'
url = f'https://api.github.com/repos/{owner}/{repo}/commits?per_page=100'  # limit 10 commits per request

headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

start_date = datetime(2024, 1, 1)  # time limit 2024
end_date = datetime(2024, 6, 6)  

params = {
    'since': start_date.isoformat() + 'Z',  #UTC timezone
    'until': end_date.isoformat() + 'Z'      
}

df24 = pd.DataFrame(columns=['Email', 'Timestamp', 'Company']) # initialise datset

def get_commits(url):    #function get info
    global commit_count
    global df24

    while url:
        response = requests.get(url, headers=headers, params=params)  # parameter:timelimit
        
        try:
            response.raise_for_status()
            commits = response.json()

            for commit in commits:
                author_url = commit['author']['url'] if commit['author'] else None
                email = commit['commit']['author']['email']
                timestamp = commit['commit']['author']['date']
                company = None

                # if there is author: get info
                if author_url:
                    user_response = requests.get(author_url, headers=headers)
                    if user_response.status_code == 200:
                        user_data = user_response.json()
                        company = user_data.get('company')  

                # add to df
                df24.loc[commit_count] = [email, timestamp, company]
                commit_count += 1

            # check if there is a next page
            url = response.links.get('next', {}).get('url')

            # waiting time between requests
            time.sleep(1)

        except requests.exceptions.HTTPError as err:
            print(f'HTTP Error: {err}')
            break


get_commits(url)

df24

In [ ]:
df24

In [ ]:
# GitHub API Token
TOKEN = ''  # Ersetze dies durch dein API-Token

# Repository-Informationen
owner = 'NixOS'
repo = 'nixpkgs'
url = f'https://api.github.com/repos/{owner}/{repo}/commits?per_page=100'

headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

start_date = datetime(2024, 1, 1)  # Zeitlimit für 2024
end_date = datetime.utcnow()  

params = {
    'since': start_date.isoformat() + 'Z',  # Startdatum
    'until': end_date.isoformat() + 'Z'      # Enddatum
}

# DataFrame nur für die Unternehmensnamen initialisieren
df_companies = pd.DataFrame(columns=['Timestamp', 'Company'])  # E-Mail-Spalte entfernt


In [ ]:
def get_commits(url):
    results = []  # Liste zur Speicherung der Ergebnisse
    while url:
        response = requests.get(url, headers=headers, params=params)
        print(f'Sending request to: {url}')  # Debugging
        print(f'Status Code: {response.status_code}')  # Debugging
        
        try:
            response.raise_for_status()
            commits = response.json()
            print(f'Found {len(commits)} commits')  # Debugging
            
            if not commits:
                print('No commits found for this request.')
                break

            for commit in commits:
                timestamp = commit['commit']['author']['date']  # Zeitstempel speichern
                company = None  # Unternehmensname initialisieren
                author_url = commit['author']['url'] if commit['author'] else None

                # Wenn ein Author-URL vorhanden ist, versuche, die Unternehmensdaten abzurufen
                if author_url:
                    user_response = requests.get(author_url, headers=headers)
                    if user_response.status_code == 200:
                        company = user_response.json().get('company')  # Unternehmensname abrufen

                # Ergebnis zur Liste hinzufügen
                results.append([timestamp, company if company else 'None'])

            # Überprüfen, ob es eine nächste Seite gibt
            url = response.links.get('next', {}).get('url')
            time.sleep(1)  # Wartezeit zwischen den Anfragen

        except requests.exceptions.HTTPError as err:
            print(f'HTTP Error: {err}')
            break

    # Konvertiere die Ergebnisse in einen DataFrame
    global df_companies  # Setze die globale DataFrame-Variable
    df_companies = pd.DataFrame(results, columns=['Timestamp', 'Company'])

# Aufrufen der Funktion
get_commits(url)

# Ausgabe des DataFrames
print(df_companies)

In [ ]:
df24.duplicated().sum()

In [ ]:
df24= df24.drop_duplicates()

In [ ]:
df24.describe()

In [ ]:
df24['Company'].unique()

In [ ]:
df24['Company'] = df24['Company'].str.replace('@', '', regex=False) #performs simple, direct string replacement(default r=True)
df24['Company'] = df24['Company'].str.strip()

In [ ]:
df24['Company'].unique()